# Notebook exploratório

- Artigo: Train one get one free
- Link: https://arxiv.org/abs/1903.12431

## 0.1 Inicializando o GloVe

In [170]:
import torchtext as tt

In [171]:
glove = tt.vocab.GloVe('6B', dim=50)
glove_vocab = set(glove.stoi.keys())

## 1.1 Carregando dados de uma das bases para exemplo

In [172]:
import pandas as pd

In [173]:
eclipse = pd.read_csv('../../data/processed/eclipse.csv', index_col='bug_id')
eclipse.head()

,short_desc
bug_id,
1,Usability issue with external editors (1GE6IRL)
2,Opening repository resources doesn't honor typ...
3,Sync does not indicate deletion (1GIEN83)
4,need better error message if catching up over ...
5,ISharingManager sharing API inconsistent (1GAU...


## 1.2 Preprocessamento dos dados

In [174]:
remove_nonalpha = lambda word: ''.join([x for x in word if str.isalpha(x) or str.isnumeric(x) or x == ' '])

short_desc = eclipse['short_desc']
short_desc = short_desc.apply(str.lower)
short_desc = short_desc.apply(remove_nonalpha)
short_desc

bug_id
1             usability issue with external editors 1ge6irl
2         opening repository resources doesnt honor type...
3                   sync does not indicate deletion 1gien83
4         need better error message if catching up over ...
5          isharingmanager sharing api inconsistent 1gaul8h
                                ...                        
424774          rwt browserwindowhandler doesnt work on osx
424775    invalid unused imports warnings and invalid or...
424776                         spinner allows any character
424778    allow to ignore builtin priorities of pluggabl...
424779                          syntax problem not reported
Name: short_desc, Length: 334422, dtype: object

## 1.3 Explorando vocabulário das descrições

In [175]:
from collections import defaultdict

In [176]:
base_freq = defaultdict(int)
for sentence in short_desc:
    for word in sentence.split():
        base_freq[word] += 1
base_vocab = set(base_freq.keys())
len(base_vocab)

111313

## 1.4 Explorando diferenças entre vocabulário do GloVe e das descrições

In [177]:
print("Número de palavras distintas: ", len(base_vocab - glove_vocab))

Número de palavras distintas:  94763


## 1.5 Removendo palavras menos frequentes do vocabulário

In [178]:
MIN_FREQ = 3

base_freq = {key: value for key, value in base_freq.items() if value > MIN_FREQ}
base_freq
base_vocab = set(base_freq.keys())
print("Número de palavras do vocabulário: ", len(base_vocab))
print("Número de palavras compartilhadas entre os dois vocabulários: ", len(base_vocab & glove_vocab))
print("Número de palavras distintas: ", len(base_vocab - glove_vocab))

Número de palavras do vocabulário:  17026
Número de palavras compartilhadas entre os dois vocabulários:  9193
Número de palavras distintas:  7833


## 1.6 Obtendo pesos das palavras existentes no vocabulário do GloVe

* Obtendo palavras que estão dentro do vocabulário do GloVe
* Extraindo o STOI das palavras que estão apenas no vocabulário
* Obtendo os vetores correspondentes as palavras pertencentes ao vocabulário
* Gerando um novo STOI iniciado de 0 usando apenas as palavras existentes no vocabulário

In [179]:
import torch

In [180]:
in_vocab = base_vocab & glove_vocab
in_size = len(in_vocab)

in_stoi = {x: glove.stoi[x] for x in in_vocab}
in_stoi = {k: v for k, v in sorted(in_stoi.items(), key=lambda x:x[1])}
in_vectors = glove.vectors[list(in_stoi.values())]
in_stoi = {key: value for key, value in zip(in_stoi.keys(), range(in_size))}

## 1.7 Gerando pesos para palavras fora do vocabulário do GloVe

* Obtem as palavras que não estão no vocabulário
* Gera um vetor de zeros com o shape `[n_palavras_fora_vocab, shape_do_glove]`
* Utiliza o vetor de zeros pra gerar um tensor que segue distribuição normal
* Gera um STOI para continuar o STOI anterior

In [181]:
glove_shape = glove.vectors.shape[-1]

out_vocab = sorted(list(base_vocab - glove_vocab))
out_size = len(out_vocab)

zeros = torch.zeros((out_size, glove_shape))
out_vectors = torch.normal(mean=zeros, std=1)

out_stoi = {key: value for key, value in zip(out_vocab, range(in_size, in_size + out_size))}

## 1.8 Unindo representações

In [182]:
base_vectors = torch.cat((in_vectors, out_vectors))
base_stoi = {**in_stoi, **out_stoi}

## 2.0 Implementando rede